项目名称：大单研究

项目描述：《少年三国志》作为果盘平台最为重点的游戏之一，每月将产生超过一千万的流水，该游戏采用了长尾市场最为普遍的“首续差”方式获取玩家和赢得利润，具体而言便是做折扣，通过低折首充（第一笔充值）获取用户，高折续充获取利润。长期以来，该游戏虽然产生较高流水，但整体利润却不高，每月仅提供20万左右的盈利，造成利润低的原因是什么？是否存在用户首充金额巨大，然而续充却较为稀少？是否有必要对于高额首充行为进行限制？

In [8]:
import numpy as np
import pandas as pd
import datetime
import os
os.chdir('D:/工作/桌面全部内容/项目/大单研究/少三安卓/')
pd.set_option('display.float_format',lambda x : '%.2f' % x)

首先我们将用户通过首充的金额进行分类，根据首充金额<5000,5000到8000,8000到10000,10000分成四类用户，查看目前用户的首充续充金额情况。是否存在用户首充金额巨大，然而续充却较为稀少的现象？

In [10]:
#数据读取与合并

data = pd.read_csv('公会发放果币卡记录1571897548.csv',engine = 'python')[['用户uin','代金券金额','消耗果粒',
                                                                 '发放时间','公会ID','是否首充']]

data['发放时间'] = pd.to_datetime(data['发放时间'])
data['发放月份'] = data['发放时间'].map(lambda x: x.month)        #提取月份

data.head()

,用户uin,代金券金额,消耗果粒,发放时间,公会ID,是否首充,发放月份
0,119476100,6.00,2.88,2019-10-23 23:54:23,68258,否,10
1,121539408,6.00,2.88,2019-10-23 23:53:26,68258,否,10
2,129351982,6.00,0.00,2019-10-23 23:50:26,82282,否,10
3,116134078,648.00,0.00,2019-10-23 23:48:51,78214,否,10
4,116134078,648.00,0.00,2019-10-23 23:48:37,78214,否,10


In [42]:
#构建函数计算用户三个月内的首续充情况

def Nusers_pay_thismonth(df):
    m = []
    for i in range(data['发放月份'].max()+1)[1:]:
        df_sum = df.loc[df['发放月份'] == i]
        df_sump = df_sum.groupby('用户uin').sum()
        df_first = df_sum.loc[df['是否首充'] == '是']
        df_last = pd.merge(df_first,df_sump,left_on = '用户uin',right_index = True)[['用户uin','代金券金额_x',
                                                                                    '代金券金额_y','发放月份_x',]]
        df_last.rename(columns = {'代金券金额_x':str(i)+'月'+'首充','代金券金额_y':str(i)+'月'+'总值'},inplace = True)
        df_last.sort_values(df_last.columns[1],ascending = False,inplace = True)
        
        #Feb
        df_more1 = df.loc[df['发放月份']==i+1]
        df_more1p = df_more1.groupby('用户uin').sum()
        df_last = pd.merge(df_last,df_more1p,left_on = '用户uin',right_index = True,how='left')[['用户uin',df_last.columns[1],
                                                                                    df_last.columns[2],'代金券金额']]
        df_last.rename(columns = {'代金券金额':str(i+1)+'月'+'总值'},inplace = True)
        
        #Mar
        df_more2 = df.loc[df['发放月份']==i+2]
        df_more2p = df_more2.groupby('用户uin').sum()
        df_last = pd.merge(df_last,df_more2p,left_on = '用户uin',right_index = True,how='left')[['用户uin',df_last.columns[1],
                        df_last.columns[2],df_last.columns[3],'代金券金额']]
        df_last.rename(columns = {'代金券金额':str(i+2)+'月'+'总值'},inplace = True)
        
        m.append(df_last)
    return(m)


#1月用户数据
def f1(lst):
    m = []
    for df in lst:
        cut = pd.cut(df[df.columns[1]],[0,5000,8000,10010,9999999999],right = False)
        df['区间'] = cut.values
        df = df.groupby('区间').sum()
        df['人数'] = cut.value_counts()
        df.fillna(0,inplace = True)
        m.append(df)
    return(m)

month1 = f1(Nusers_pay_thismonth(data))[0].iloc[:,1:]
month1['综合利润'] = month1['1月首充']*-0.055 + ((month1['1月总值'] - month1['1月首充']) + month1['2月总值'] + month1['3月总值'])*0.075
month1

,1月首充,1月总值,2月总值,3月总值,人数,综合利润
区间,,,,,,
"[0, 5000)",1396476.10,2717940.10,1354094.00,529914.00,3573,163604.21
"[5000, 8000)",217889.00,377689.00,233751.00,66629.00,38,22529.60
"[8000, 10010)",439037.00,789006.00,413775.00,256631.00,49,52381.09
"[10010, 9999999999)",596465.00,776888.00,333057.00,140897.00,31,16272.70


In [44]:
#2月用户数据
month2 = f1(Nusers_pay_thismonth(data))[1].iloc[:,1:]
month2['综合利润'] = month2['2月首充']*-0.055 + ((month2['2月总值'] - month2['2月首充']) + month2['3月总值'] + month2['4月总值'])*0.075
month2

,2月首充,2月总值,3月总值,4月总值,人数,综合利润
区间,,,,,,
"[0, 5000)",1031829.00,1791204.00,1057684.00,360479.00,1978,106564.76
"[5000, 8000)",144887.00,236650.00,157858.00,31575.00,25,13120.92
"[8000, 10010)",375494.00,592461.00,370406.00,157167.00,43,35188.33
"[10010, 9999999999)",952291.00,1076522.00,470564.00,208867.00,31,7898.64


上面我们简单的抽取了1月和2月的用户首续充情况，我们可以明显发现利润的主要来源是低档位的用户，高档位用户（首充10000以上）带来的利润是最低的

大单用户（首充大于10000）所带来的利润最低，从2019年7月起将对该批用户实行限制，禁止用户首充高于10000，大单限制实行后，整体流水下滑不足20%，用户首充档位有下移趋势，整体利润翻倍，上涨至40万每月